In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
#from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama

embeddings = OllamaEmbeddings(model='nomic-embed-text:v1.5')

path_books = "./vector-store/books"
db = FAISS.load_local(path_books, embeddings, allow_dangerous_deserialization=True)

retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = False
retriever.search_kwargs["k"] = 5

In [2]:
question = "O que é mais difícil para o obreiro cristão?"

In [3]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


prompt = PromptTemplate.from_template(
    """ 
    <s>[INST] Você é um assistente técnico bom em pesquisar documentos. 
    Se você não tiver uma resposta com base nas informações fornecidas, diga-o. [/INST] </s>
    [INST] {input}
           Contexto: {context}
           Resposta em português:
    [/INST]
"""
)
llm = Ollama(model="mistral:7b", temperature=0)
document_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, document_chain)

result = chain.invoke({"input": question})

#print(result)

sources = []
for doc in result["context"]:
    sources.append(
        {"source": doc.metadata["source"], "page_content": doc.page_content}
    )

response_answer = {"answer": result["answer"], "sources": sources}


In [ ]:
response_answer["answer"]

' I. Introdução\n\nNesta seção, vamos explorar a importância da obediência na escola dominical e como ela reflete o amor de Cristo por crianças. A obediência é um ato fundamental de um discípulo de Jesus, e na escola dominical, ela se manifesta através do trabalho realizado pelos professores e voluntários em nome de Cristo.\n\nII. Motivações para a Obediência na Escola Dominical\n\n1. A obediência na escola dominical é uma expressão de amor por Jesus, Senhor e Mestre.\n2. A obediência na escola dominical é um ato de cumprimento do grande mandamento: "Ide por todo o mundo, e pregai o evangelho a toda criatura" (Mc 16.15).\n3. A obediência na escola dominical é uma resposta às palavras de Jesus: "Deixai vir a mim as crianças e não as impeçais" (Mt 19.14) e "Cuida dos meus cordeiros" (Jo 21.15).\n4. A obediência na escola dominical é uma maneira de cumprir a Grande Comissão: "Pregai o evangelho a toda criatura" (Mc 16.15).\n\nIII. Benefícios da Obediência na Escola Dominical\n\n1. A obedi

In [ ]:
response_answer["sources"]

[{'source': './books/import/Conselhos para obreiros - Charles H Spurgeon.pdf',
  'page_content': 'meu servo Moisés te ordenou”. A maior proeza da vida cristã é obedecer a Cristo. É um feito que\nnunca será realizado em sua totalidade por nenhum homem, a não ser que aprenda a lei da fé, que\nseja levado a descansar em Cristo e a avançar no caminho da obediência por meio de uma força\nque não é dele, mas sim a que recebeu da obra do Espírito Santo que reside em nós. O mundo\nacha que obediência é algo ruim e fala da rebelião no sentido de liberdade. Já ouvi homens\ndizerem: “Serei eu o meu mestre; e seguirei o meu desejo”. Viver e pensar livremente parece ser\na  glória  mundana.  Mas  se  o  mundo  tivesse  senso  suficiente  para  condenar  sua  loucura,\nconsiderando-se a inquestionável quantidade de provas nesse sentido, não seria difícil provar que\num  difamador  da  obediência  é  um  tolo.  Veja  a  própria  lei  marcial  do  mundo.  Quem  é\nconsiderado o mais corajoso e o melho

In [3]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Ollama

# RAG
template = """Responda a pergunta abaixo com base no contexto fornecido e citando o contexto utilizado. 
Ignore trechos do contexto que são sumários, índices, ou referências a outras partes do texto.

Contexto: {context}

Questão: {question}

Resposta em português:
"""

prompt = ChatPromptTemplate.from_template(template)

llm = Ollama(model="mistral:7b", temperature=0)

final_rag_chain = (
    {"context": retriever, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

' Responder com precisão a essa pergunta pode variar de um indivíduo para outro, pois cada obreiro cristão enfrenta desafios e dificuldades únicas na sua caminhada espiritual. No entanto, podemos identificar algumas dificuldades comuns que muitos obreiros cristãos enfrentam:\n\n1. Manter a fé e a dedicação: Obreiros cristãos são chamados a servir o Senhor em diferentes áreas da vida, mas à medida que passam mais tempo no ministério, eles podem se cansar ou perder a fervor inicial. Mantendo uma forte fé e dedicação ao ministério pode ser difícil, especialmente quando enfrentamos dificuldades pessoais ou quando recebemos críticas ou insatisfação por parte de outros.\n2. Balançar a vida pessoal e profissional: Os obreiros cristãos geralmente têm uma vida complexa que inclui sua família, amigos, saúde e tempo livre, além de suas responsabilidades no ministério. Achar equilíbrio entre essas áreas pode ser difícil e exigir muita paciência e oração.\n3. Encounterar oposição e críticas: Obreir

In [4]:
# Multi Query: Different Perspectives
template = """Você é um assistente de modelo de linguagem de IA. Sua tarefa é gerar cinco
diferentes versões da pergunta do usuário para recuperar documentos relevantes de um vetor
base de dados. Ao gerar múltiplas perspectivas sobre a pergunta do usuário, seu objetivo é ajudar
o usuário supera algumas das limitações da busca por similaridade baseada em distância.
Forneça essas perguntas alternativas separadas por novas linhas na língua da pergunta original. 
Pergunta original: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)



# LLM
llm = Ollama(model="mistral:7b")

generate_queries = (
    prompt_perspectives 
    | llm #ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [5]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve

retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

/home/cristian/miniconda3/envs/ollama/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


11

In [8]:
from operator import itemgetter
#from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Ollama

# RAG
template = """Responda a seguinte questão baseado nesse contexto:

{context}

Questão: {question}
Resposta somente em português:
"""

prompt = ChatPromptTemplate.from_template(template)

#llm = ChatOpenAI(temperature=0)
llm = Ollama(model="mistral:7b", temperature=0)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

' Determinar a equilíbrio entre trabalhar diligentemente nos negócios e fervorizar no Espírito, buscando harmonizar as duas partes. O cristão deve ser um homem justo e piedoso, que não deixa que um dever suje o outro. (Documento: page_content="Seguindo o mesmo princípio, organize seus negócios. Alguns homens de negócios agem de tal maneira que, desde a manhã até a noite, só pensam nos negócios. Tenho de repreender alguns cristãos, que não conseguem discernir quando já têm o suficiente. Exageram no trabalho e se gastam pelas empresas, mas deveriam gastar a sua saúde pela sua alma, pois não têm mais necessidade de ganho. Mas ainda pensam em iniciar mais alguma coisa que afastará todas as opportunidades de servir à causa de Cristo e lhes tirará o tempo necessário para a reflexão, e isso trará aridez e pobreza à sua alma. Há extremos; mas o verdadeiro cristão é diligente nos negócios e, ao mesmo tempo, fervoroso no Espírito, buscando sempre harmonizar as duas partes. O crente deveria ser c

In [7]:
relevant_documents = retriever.invoke({"question": question})
for row in relevant_documents:
    page_content = row.page_content
    source = row.metadata['source']
    page = row.metadata['page']

    

    # Agora você pode fazer o que quiser com esses dados.
    # Por exemplo, você pode imprimi-los:
    print(f"""
Page Content: 
{page_content.replace('	',' ')}

Source: 
{source} 

Page:
{page}

""")


Page Content: 
12
Na escola dominical
rabalho na escola dominical é beneficência. E como poderia ser de outra forma, se é um
ato de obediência? Creio que você começou a fazer esse trabalho porque chama Jesus
de seu Mestre e Senhor e deseja cumprir o grande mandamento: “Ide por todo o mundo,
e pregai o evangelho a toda criatura” (Mc 16.15). Você vê as crianças como criaturas, criaturas
decaídas, mas ainda coisinhas adoráveis, cheias de vigor, vida e alegria. Você as vê como parte
da humanidade e conclui, imediatamente, que a ordem de seu Mestre se aplica a elas. Você não é
como os discípulos que queriam afastá-las, pois aprendeu com o erro deles e se lembra das
palavras do Mestre: “Deixai vir a mim as crianças e não as impeçais” (Mt 19.14). Você sabe,
também, que ele conferiu força à boca de bebês e crianças de peito, por causa do inimigo; de
modo que tem certeza de que ele incluiu os pequeninos na Grande Comissão, quando disse:
“Pregai o evangelho a toda criatura” (Mc 16.15).T
Você es